# Imports

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import metrics
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
# this adds the src folder in the sys path, where the metric_utils.py file is
# not needed if this notebook is in the same folder, but uncomment to access from the data subfolders
sys.path.append( '..' )
from metric_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Result path

In [2]:
dataset_name = 'ucberkeley hate speech'
model_name = 'bert'

# change to ../../results/{dataset_name} when using inside one of the data subfolders
result_folder = f'../../results/{dataset_name}'
test_csv_filepath = os.path.join(result_folder, 'test.csv')

model_folder = os.path.join(result_folder, model_name) # for this particular model
normal_folder = os.path.join(model_folder, 'normal')
dp_folder = os.path.join(model_folder, 'epsilon 3.0')

result_filepath = os.path.join(normal_folder, 'results.csv')
dp_result_filepath = os.path.join(dp_folder, 'results.csv')

# Dataset
The dataset can be preprocessed from the original dataset to be used here. To simplify things, I saved the preprocessed datasets during the tokenizing process as csv files and then downloaded them in the corresponding dataset folder of [`result`](../results/) directory.

You can recreate the processed datasets using the tokenize notebooks for that particular dataset. That would give you train, test and validation csv files as well as the tokenized data.

In [3]:
result = pd.read_csv(result_filepath)
dp_result = pd.read_csv(dp_result_filepath)
df = pd.read_csv(test_csv_filepath)

In [4]:
df.head(2)

,comment_id,text,target_gender_men,target_gender_women,target_race_white,target_race_black,labels
0,41012,I don't want to see America treated like a garbage can. Luv from Canada.,0,0,0,0,0
1,31056,@mike_pence @realDonaldTrump You're a Christian of convenience. You're a fake,0,0,0,0,0


## Split test result

In [5]:
# only calculate test result
result = result[result['split']=='test']
dp_result = dp_result[dp_result['split']=='test']

# drop split column
result.drop(columns=['split'], inplace=True)
dp_result.drop(columns=['split'], inplace=True)

## Prepare test data

In [6]:
# result has id column which is the same as the text ids from raw dataset
raw_id_column = 'comment_id'
df.rename({raw_id_column: id_column}, axis=1, inplace=True)
df.drop(columns=['labels'], inplace=True)

## Merge

In [7]:
result = result.merge(df, on=id_column, how='inner').reset_index(drop=True)
dp_result = dp_result.merge(df, on=id_column, how='inner').reset_index(drop=True)

In [8]:
result.head()

,id,labels,probs,text,target_gender_men,target_gender_women,target_race_white,target_race_black
0,41012,0,0.502331,I don't want to see America treated like a garbage can. Luv from Canada.,0,0,0,0
1,31056,0,0.214350,@mike_pence @realDonaldTrump You're a Christian of convenience. You're a fake,0,0,0,0
2,17058,0,0.370149,Jesus just has his holy reverse card that he uses on the microbes. Does that girl have a holy reverse? I thot not.,0,0,0,0
3,4023,0,0.083863,"Why? It's never even hinted that being gay or bi means you end up in the Bad Place, if that's what you meant?",0,0,0,0
4,3792,0,0.673117,I'm half Jewish you fucking idiot. Eat a dick,0,0,0,0


# Evaluation

## Convert probability to prediction

In [9]:
result[prediction_column] = result[probability_column] >=0.5
dp_result[prediction_column] = dp_result[probability_column] >=0.5

## Identity groups

In [10]:
group_map = {
    'gender': {
        'unprivileged':['target_gender_women'],
        'privileged':['target_gender_men']
    },
    'race': {
        'unprivileged':['target_race_black'],
        'privileged': ['target_race_white']
    }
}

identities = []
for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    for subgroup_key in subgroup_map.keys():
        identities.extend(subgroup_map[subgroup_key])

print(identities)

['target_gender_women', 'target_gender_men', 'target_race_black', 'target_race_white']


### Binarize identity and target columns

In [11]:
result = binarize(result, [target_column] + identities)
dp_result = binarize(dp_result, [target_column] + identities)

## Bias

In [12]:
bias_results = {
   'fairness_metrics': ['demographic parity', 'Equality of Opportunity (w.r.t y = 1)',
   'Equality of Opportunity (w.r.t y = 0)', 'Equality of Odds', 'unprotected-accuracy',
   'protected-accuracy', 'accuracy']
}

for group_key in group_map.keys():
   subgroup_map = group_map[group_key]
   privileged_group = subgroup_map['privileged']
   unprivileged_group = subgroup_map['unprivileged']

   bias_results[group_key] = calculate_bias(result, privileged_group, unprivileged_group)
   bias_results[group_key+'_DP'] = calculate_bias(dp_result, privileged_group, unprivileged_group)

bias_results = pd.DataFrame(bias_results) 
bias_results

,fairness_metrics,gender,gender_DP,race,race_DP
0,demographic parity,0.468336,0.611193,0.517110,0.543726
1,Equality of Opportunity (w.r.t y = 1),0.923097,0.979236,0.699580,0.633739
2,Equality of Opportunity (w.r.t y = 0),0.933718,0.971290,0.922979,0.927503
3,Equality of Odds,0.928407,0.975263,0.811279,0.780621
4,unprotected-accuracy,0.756967,0.734284,0.802680,0.780755
5,protected-accuracy,0.796760,0.773196,0.787072,0.764259
6,accuracy,0.776863,0.753740,0.794876,0.772507


In [13]:
bias_results.to_csv(os.path.join(dp_folder, 'bias.csv'), index=False)

## Overall metrics

In [14]:
overall_results = {
    'metrics': ['auc', 'accuracy', 'f1_score', 'precision', 'recall', 'false positive rate']
}

for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    privileged_group = subgroup_map['privileged']
    unprivileged_group = subgroup_map['unprivileged']

    privileged_group_name = ','.join(privileged_group)
    unprivileged_group_name = ','.join(unprivileged_group)

    overall_results[privileged_group_name] = calculate_metrics(result, privileged_group)
    overall_results[privileged_group_name + '_DP'] = calculate_metrics(dp_result, privileged_group)

    overall_results[unprivileged_group_name] = calculate_metrics(result, unprivileged_group)
    overall_results[unprivileged_group_name + '_DP'] = calculate_metrics(dp_result, unprivileged_group)

overall_results['Total'] = calculate_metrics(result, [])
overall_results['Total_DP'] = calculate_metrics(dp_result, [])

overall_results = pd.DataFrame(overall_results) 
overall_results

,metrics,target_gender_men,target_gender_men_DP,target_gender_women,target_gender_women_DP,target_race_white,target_race_white_DP,target_race_black,target_race_black_DP,Total,Total_DP
0,auc,0.823891,0.790713,0.816284,0.782050,0.788782,0.737971,0.877634,0.858890,0.858167,0.821560
1,accuracy,0.796760,0.773196,0.756967,0.734284,0.787072,0.764259,0.802680,0.780755,0.806742,0.789628
2,f1_score,0.594118,0.524691,0.635569,0.552402,0.540984,0.420561,0.772472,0.730539,0.637484,0.562073
3,precision,0.619632,0.578231,0.624046,0.615572,0.611111,0.576923,0.759669,0.767296,0.665088,0.665814
4,recall,0.570621,0.480226,0.647525,0.500990,0.485294,0.330882,0.785714,0.697143,0.612080,0.486301
5,false positive rate,0.123506,0.123506,0.189788,0.152216,0.107692,0.084615,0.184713,0.157113,0.118449,0.093802


In [15]:
overall_results.to_csv(os.path.join(dp_folder, 'overall_results.csv'), index=False)

# Batch benchmarking
Benchmark all models and privacy budgets

In [16]:
for model_name in ['bert', 'distilbert']:
    model_folder = os.path.join(result_folder, model_name) # for this particular model
    normal_folder = os.path.join(model_folder, 'normal')
    result_filepath = os.path.join(normal_folder, 'results.csv')

    result = pd.read_csv(result_filepath)
    result = result[result['split']=='test']
    # drop split column
    result.drop(columns=['split'], inplace=True)

    result = result.merge(df, on=id_column, how='inner').reset_index(drop=True)
    result[prediction_column] = result[probability_column] >=0.5
    result = binarize(result, [target_column] + identities)

    
    for epsilon in [3.0, 6.0, 9.0]:
        dp_folder = os.path.join(model_folder, f'epsilon {epsilon}')
        dp_result_filepath = os.path.join(dp_folder, 'results.csv')
        dp_result = pd.read_csv(dp_result_filepath)

        # only calculate test result
        dp_result = dp_result[dp_result['split']=='test']
        dp_result.drop(columns=['split'], inplace=True)
        dp_result = dp_result.merge(df, on=id_column, how='inner').reset_index(drop=True)
        
        dp_result[prediction_column] = dp_result[probability_column] >=0.5
        dp_result = binarize(dp_result, [target_column] + identities)

        bias_results = {
        'fairness_metrics': ['demographic parity', 'Equality of Opportunity (w.r.t y = 1)',
        'Equality of Opportunity (w.r.t y = 0)', 'Equality of Odds', 'unprotected-accuracy',
        'protected-accuracy', 'accuracy']
        }

        for group_key in group_map.keys():
            subgroup_map = group_map[group_key]
            privileged_group = subgroup_map['privileged']
            unprivileged_group = subgroup_map['unprivileged']

            bias_results[group_key] = calculate_bias(result, privileged_group, unprivileged_group)
            bias_results[group_key+'_DP'] = calculate_bias(dp_result, privileged_group, unprivileged_group)

        bias_results = pd.DataFrame(bias_results) 
        bias_results.round(3).to_csv(os.path.join(dp_folder, 'bias.csv'), index=False)


        overall_results = {
            'metrics': ['auc', 'accuracy', 'f1_score', 'precision', 'recall', 'false positive rate']
        }

        for group_key in group_map.keys():
            subgroup_map = group_map[group_key]
            privileged_group = subgroup_map['privileged']
            unprivileged_group = subgroup_map['unprivileged']

            privileged_group_name = ','.join(privileged_group)
            unprivileged_group_name = ','.join(unprivileged_group)

            overall_results[privileged_group_name] = calculate_metrics(result, privileged_group)
            overall_results[privileged_group_name + '_DP'] = calculate_metrics(dp_result, privileged_group)

            overall_results[unprivileged_group_name] = calculate_metrics(result, unprivileged_group)
            overall_results[unprivileged_group_name + '_DP'] = calculate_metrics(dp_result, unprivileged_group)

        overall_results['Total'] = calculate_metrics(result, [])
        overall_results['Total_DP'] = calculate_metrics(dp_result, [])

        overall_results = pd.DataFrame(overall_results) 
        overall_results.columns = [col.replace('target_', '') for col in overall_results.columns]
        overall_results.round(3).to_csv(os.path.join(dp_folder, 'overall_results.csv'), index=False)